In [1]:
# add local directory
import sys
sys.path.append('../')
print(sys.version)
print(*sys.path, sep='\n')

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
/home/lukec/workspace/aad/examples
/home/lukec/.vscode/extensions/ms-python.python-2020.3.71113/pythonFiles
/home/lukec/.vscode/extensions/ms-python.python-2020.3.71113/pythonFiles/lib/python
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/home/lukec/venv/lib/python3.6/site-packages
/home/lukec/workspace/path_demo
/home/lukec/.local/lib/python3.6/site-packages
/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/home/lukec/venv/lib/python3.6/site-packages/IPython/extensions
/home/lukec/.ipython
../


In [2]:
import os

import torch
import numpy as np

from aad.datasets import DATASET_LIST, DataContainer
from aad.basemodels import ModelContainerPT, MnistCnnCW
from aad.attacks import SaliencyContainer

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
DATA_ROOT = os.path.join('..', 'data')
BATCH_SIZE = 128

# image datasets: {'MNIST', 'CIFAR10', 'SVHN'}
# quantitative datasets: {'BankNote', 'BreastCancerWisconsin', 'HTRU2', 'Iris', 'WheatSeed'}
NAME = 'MNIST'
print(f'Starting {NAME} data container...')
print(DATASET_LIST[NAME])

dc = DataContainer(DATASET_LIST[NAME], DATA_ROOT)
dc(size_train=0.8, normalize=True)

num_features = dc.dim_data[0]
num_classes = dc.num_classes
print('Features:', num_features)
print('Classes:', num_classes)

## model in {BCNN, IrisNN, MnistCnnCW}
# model = IrisNN(num_features, num_classes, hidden_nodes=16)  # for Iris
# model = BCNN(num_features, num_classes)
model = MnistCnnCW()
model_name = model.__class__.__name__
print('Using model:', model_name)

mc = ModelContainerPT(model, dc)
mc.fit(max_epochs=3, batch_size=BATCH_SIZE)

Starting MNIST data container...
{'name': 'MNIST', 'type': 'image', 'size': 70000, 'num_classes': 10, 'dim_data': (1, 28, 28)}
Features: 1
Classes: 10
Using model: MnistCnnCW


In [ ]:
attack = SaliencyContainer(mc)
print(attack.attack_params)

In [ ]:
n = 32

# targeted
# targets = attack.randam_targets(n, dc.num_classes, use_onehot=True)
# adv, y_adv, x_clean, y_clean = attack.generate(count=n, targets=targets)

# untargeted
adv, y_adv, x_clean, y_clean = attack.generate(count=n)
print(attack.attack_params)
print(adv.shape)
print(y_adv.shape)
print(x_clean.shape)
print(y_clean.shape)

In [ ]:
accuracy = mc.evaluate(x_clean, y_clean)
print(f'Accuracy on clean samples: {accuracy*100:.4f}%')
accuracy = mc.evaluate(adv, y_clean)
print(f'Accuracy on adversarial example: {accuracy*100:.4f}%')